In [22]:
#Librarries

In [ ]:
#!pip install requests
#!pip install pandas

In [26]:
import requests
import base64
import json
import pandas as pd
from datetime import datetime, timedelta
import time

In [38]:
#API address
url = "https://api.coingecko.com/api/v3/coins/markets"

#Mandatory_parameters
params = {
    "vs_currency": "usd",  # Required!
    "ids": "ethereum,solana,cardano,avalanche-2,chainlink"
}

#Fetching_Data
response = requests.get(url, params=params)

print("Status:", response.status_code)
print("Response:", response.json())


Status: 200
Response: [{'id': 'ethereum', 'symbol': 'eth', 'name': 'Ethereum', 'image': 'https://coin-images.coingecko.com/coins/images/279/large/ethereum.png?1696501628', 'current_price': 2610.73, 'market_cap': 315222943824, 'market_cap_rank': 2, 'fully_diluted_valuation': 315222943824, 'total_volume': 25170038545, 'high_24h': 2611.31, 'low_24h': 2498.01, 'price_change_24h': 72.58, 'price_change_percentage_24h': 2.85943, 'market_cap_change_24h': 8772633976, 'market_cap_change_percentage_24h': 2.86266, 'circulating_supply': 120727923.2786977, 'total_supply': 120727923.2786977, 'max_supply': None, 'ath': 4878.26, 'ath_change_percentage': -46.47306, 'ath_date': '2021-11-10T14:24:19.604Z', 'atl': 0.432979, 'atl_change_percentage': 602974.1873, 'atl_date': '2015-10-20T00:00:00.000Z', 'roi': {'times': 32.6254683708478, 'currency': 'btc', 'percentage': 3262.5468370847793}, 'last_updated': '2025-05-16T10:04:44.742Z'}, {'id': 'solana', 'symbol': 'sol', 'name': 'Solana', 'image': 'https://coin-

In [32]:
#Checking_Schema
data = response.json()

# Convert to DataFrame
df = pd.DataFrame(data)

# View selected columns
print(df[['name', 'symbol', 'current_price', 'market_cap', 'total_volume', 'ath', 'atl']])


        name symbol  current_price    market_cap  total_volume      ath  \
0   Ethereum    eth    2561.340000  309106094205   23002685218  4878.26   
1     Solana    sol     171.510000   89158418005    4835111818   293.31   
2    Cardano    ada       0.771921   27839048574    1005930686     3.09   
3  Chainlink   link      16.350000   10741209333     534885784    52.70   
4  Avalanche   avax      23.880000    9999535888     535649740   144.96   

        atl  
0  0.432979  
1  0.500801  
2  0.019253  
3  0.148183  
4  2.800000  


# Handling API Timing Issues in Historical Crypto Data Collection.
#### When collecting 30 days of historical data using CoinGecko's /history endpoint, we ran into a key issue (HTTP 429 — Too Many Requests).
#### Since CoinGecko limits how fast you can make API calls, and we had 180 API calls (6 coins × 30 days).

### Solution:
#### 1- We Used "time.sleep(2)" between requests ==> Slows down the pace to avoid hitting the limit.
#### 2- If we get rate-limited, we wait 60 seconds, then retry (up to 3 times)



In [39]:
#Set parameters
coins = ['bitcoin', 'ethereum', 'solana', 'cardano', 'avalanche-2', 'chainlink']
currency = 'gbp'
days = 30
headers = {'User-Agent': 'CryptoDashboardADF/1.0'}
MAX_RETRIES = 3   #Retry up to 3 times on error 429

all_data = []   #Collect all results here

#Loop through each coin for each the past 30 days
for coin in coins:
    for i in range(days):
        #Format date as DD-MM-YYYY (required by CoinGecko's history endpoint)
        date = (datetime.now() - timedelta(days=i)).strftime('%d-%m-%Y')
        
        #Call CoinGecko's history endpoint (one API call per day per coin)
        url = f"https://api.coingecko.com/api/v3/coins/{coin}/history"
        params = {'date': date, 'localization': 'false'}

        #Retry mechanism to handle rate limiting (429)
        for attempt in range(MAX_RETRIES):
            response = requests.get(url, params=params, headers=headers)
            if response.status_code == 200:
                break   #Success
         
            elif response.status_code == 429:
                print(f" Rate limited for {coin} on {date}. Waiting 60 seconds...")
                time.sleep(60)  #Wait a full minute, then retry
            else:
                print(f" Failed for {coin} on {date}: {response.status_code}")
                break  # Other error — do not retry

         #If respons's OK, extract and store the data
        if response.status_code == 200:
            data = response.json()
            market = data.get('market_data', {})
            if market:  #Only proceed if market data is available
                all_data.append({
                    'id': coin,
                    'symbol': data.get('symbol', ''),
                    'name': data.get('name', ''),
                    'current_price': market.get('current_price', {}).get(currency),
                    'market_cap': market.get('market_cap', {}).get(currency),
                    'total_volume': market.get('total_volume', {}).get(currency),
                    'high_24h': market.get('high_24h', {}).get(currency),
                    'low_24h': market.get('low_24h', {}).get(currency),
                    'price_change_24h': market.get('price_change_24h'),
                    'price_change_percentage_24h': market.get('price_change_percentage_24h'),
                    'market_cap_change_24h': market.get('market_cap_change_24h'),
                    'market_cap_change_percentage_24h': market.get('market_cap_change_percentage_24h'),
                    'circulating_supply': market.get('circulating_supply'),
                    'total_supply': market.get('total_supply'),
                    'ath': market.get('ath', {}).get(currency),
                    'atl': market.get('atl', {}).get(currency),
                    'ath_change_percentage': market.get('ath_change_percentage', {}).get(currency),
                    'atl_change_percentage': market.get('atl_change_percentage', {}).get(currency),
                    'ath_date': market.get('ath_date', {}).get(currency),
                    'atl_date': market.get('atl_date', {}).get(currency),
                    'last_updated': f"{(datetime.now() - timedelta(days=i)).strftime('%Y-%m-%dT00:00:00')}"
                })
            else:
                print(f"No market data for {coin} on {date}")
        time.sleep(2)  #Always wait 2 seconds between requests to avoid hitting limits

# Save to JSON
df = pd.DataFrame(all_data)
df.to_json("full_crypto_history_30days.json", orient="records", indent=4)
print("Saved historical data to full_crypto_history_30days.json")


 Rate limited for bitcoin on 11-05-2025. Waiting 60 seconds...
 Rate limited for bitcoin on 06-05-2025. Waiting 60 seconds...
 Rate limited for bitcoin on 04-05-2025. Waiting 60 seconds...
 Rate limited for bitcoin on 30-04-2025. Waiting 60 seconds...
 Rate limited for bitcoin on 26-04-2025. Waiting 60 seconds...
 Rate limited for bitcoin on 21-04-2025. Waiting 60 seconds...
 Rate limited for ethereum on 16-05-2025. Waiting 60 seconds...
 Rate limited for ethereum on 11-05-2025. Waiting 60 seconds...
 Rate limited for ethereum on 09-05-2025. Waiting 60 seconds...
 Rate limited for ethereum on 05-05-2025. Waiting 60 seconds...
 Rate limited for ethereum on 01-05-2025. Waiting 60 seconds...
 Rate limited for ethereum on 26-04-2025. Waiting 60 seconds...
 Rate limited for ethereum on 20-04-2025. Waiting 60 seconds...
 Rate limited for solana on 15-05-2025. Waiting 60 seconds...
 Rate limited for solana on 13-05-2025. Waiting 60 seconds...
 Rate limited for solana on 09-05-2025. Waiting 60